# Saving Algerian Constitution 2020 into SurrealDB

## 1. Importing libraries

In [ ]:
%pip install surrealdb==1.0.3 google-genai==1.2.0 python-dotenv==1.0.1

  Attempting uninstall: openai
    Found existing installation: openai 1.55.2
    Uninstalling openai-1.55.2:
      Successfully uninstalled openai-1.55.2
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datetime import datetime
from dotenv import load_dotenv
from surrealdb import Surreal
from google import genai
import json
import os

load_dotenv(override=True)

True

## 2. Load the data

In [31]:
json_file_path = os.path.join("json", "dz_consti_2020.json")
with open(json_file_path, "r", encoding="utf-8") as f:
    constitution_data = json.load(f)

## 3. Connect to the database

In [32]:
db = Surreal(os.getenv("SURREAL_URL"))
db.signin({"username": os.getenv("SURREAL_USER"), "password": os.getenv("SURREAL_PASS")})
db.use(os.getenv("SURREAL_NAMESPACE"), os.getenv("SURREAL_DATABASE"))

print(db.version())

surrealdb-2.2.0


## 4. Embed articles and save to database

In [ ]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
embed_model_name = "embedding-001"

print(list(client))

def save_constitution():
    for article in constitution_data:
        article_text = article['article']
        title_header = article['title']
        chapter_header = article['chapter']
        # more about this on https://ai.google.dev/gemini-api/docs/embeddings#python
        response = client.models.embed_content(
            model=embed_model_name,
            contents=f"{title_header}: {chapter_header}: {article_text}",  
        )
        vector = response.embeddings[0].values

        db.insert(
            "articles",
            [{
                "text": article_text,
                "embed_vector": vector,
                "chapter": chapter_header,
                "title": title_header,
                "edition": datetime(2020, 11, 1).strftime('%Y-%m-%dT%H:%M:%SZ'),
            }]
        )


save_constitution()

[Model(name='models/chat-bison-001', display_name='PaLM 2 Chat (Legacy)', description='A legacy text-only model optimized for chat conversations', version='001', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=4096, output_token_limit=1024, supported_actions=['generateMessage', 'countMessageTokens']), Model(name='models/text-bison-001', display_name='PaLM 2 (Legacy)', description='A legacy model that understands text and generates text as an output', version='001', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=8196, output_token_limit=1024, supported_actions=['generateText', 'countTextTokens', 'createTunedTextModel']), Model(name='models/embedding-gecko-001', display_name='Embedding Gecko', description='Obtain a distributed representation of a text.', version='001', endpoints=None, labels=None, tuned_model_info=Tuned

ClientError: 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/text-multilingual-embedding-002 is not found for API version v1beta, or is not supported for embedContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}

## 5. Test Search in Database

In [ ]:
# user_question = "worshiping in Algeria"
user_question = "Qu'est-ce que je peux faire dans la guerre?"

response = client.embeddings.create(
    input=user_question,
    model=embed_model_name,
)
q_vector = response.data[0].embedding

results = db.query("SELECT text, vector::similarity::cosine(embed_vector, $q_vector) AS score FROM articles ORDER BY score DESC LIMIT 10", {"q_vector": q_vector})

print(json.dumps(results, indent=2, ensure_ascii=False))

[
  {
    "score": 0.21770371660865162,
    "text": "المادة 29 : تمتنع الجزائر عن اللّجوء إلى الحرب من أجل المساس بالسّيادة المشروعة للشّعوب الأخرى وحرّيّتها. وتبذل جهدها لتسوية الخلافات الدّوليّة بالوسائل السّلميّة."
  },
  {
    "score": 0.1928271882325211,
    "text": "المادة 169 : الحقّ في الدّفاع معترف به. الحقّ في الدّفاع مضمون في القضايا الجزائيّة."
  },
  {
    "score": 0.18120915981811977,
    "text": "المادة 28 : تنتظم الطّاقة الدّفاعيّة للأمّة، ودعمها، وتطويرها، حول الجيش الوطنيّ الشّعبيّ. تتمثّل المهمّة الدّائمة للجيش الوطنيّ الشّعبيّ في المحافظة على الاستقلال الوطنيّ، والدّفاع عن السّيادة الوطنيّة. كما يضطلع بالدّفاع عن وحدة البلاد، وسلامتها التّرابيّة، وحماية مجالها البرّيّ والجوّيّ، ومختلف مناطق أملاكها البحريّة."
  },
  {
    "score": 0.1785322494551577,
    "text": "المادة 71  : الحقّ في الإضراب مُعترَف به، ويُمارَس في إطار القانون. يمكن أن يمنع القانون ممارسة هذا الحقّ، أو يجعل حدودا لممارسته في ميادين الدّفاع الوطنيّ والأمن، أو في جميع الخدمات أو الأعمال العموميّة ذا